In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from nb_104 import *

In [ ]:
path = Path('../../data/camvid')

In [ ]:
path.ls()

In [ ]:
path_lbl = path/'labels'
path_img = path/'images'

## Data

In [ ]:
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
path_lbl.ls()[:5]

In [ ]:
img_f = fnames[0]
img = open_image(img_f)
img.show(figsize=(5,5))

In [ ]:
codes = np.loadtxt(path/'codes.txt', dtype=str)
codes

In [ ]:
def get_y_fn(fn): return path_lbl/f'{fn.name[:-4]}_P.png'

In [ ]:
mask = open_mask(get_y_fn(img_f))
mask.show(figsize=(5,5), alpha=1)

In [ ]:
mask.data

## Datasets

In [ ]:
tfms = get_transforms()

In [ ]:
data = (ImageFileList.from_folder(path_img)
        .label_from_func(get_y_fn)
        .split_by_fname_file(path/'valid.txt', path_img)
        .datasets(SegmentationDataset, tfms, size=128, tfm_y=True)
        .databunch(path=path))

In [ ]:
data.path

In [ ]:
x,y = data.train_dl.one_batch()
show_xy_images(x,y,rows=3)

In [ ]:
x.shape,y.shape

In [ ]:
show_xy_images(x,y,rows=3)

In [ ]:
name2id = {v:k for k,v in enumerate(codes)}
void_code = name2id['Void']

In [ ]:
def accuracy_no_void(input, target, void_code):
    target = target.squeeze()
    mask = target != void_code
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

In [ ]:
metrics=partial(accuracy_no_void, void_code=void_code)
lr = 1e-3

In [ ]:
body = create_body(models.resnet34(True), -2)
model = models.unet.DynamicUnet(body, n_classes=len(codes)).cuda()
learn = Learner(data, model, metrics=metrics, loss_func=CrossEntropyFlat())
learn.split([model[0][6], model[1]])
learn.freeze()

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr = 1e-3

In [ ]:
learn.fit_one_cycle(6, slice(lr))

In [ ]:
learn.unfreezefreeze()

In [ ]:
learn.fit_one_cycle(6, slice(lr/100,lr))